## Scenario 3 (Azure): Team using a remote MLflow server on Azure

This scenario demonstrates connecting to a remote MLflow Tracking Server hosted on Azure (e.g., Azure VM), with a production-ready backend database (Azure Database for PostgreSQL) and optional artifact storage on Azure Blob.

High-level architecture:
- Tracking server: Azure VM (Ubuntu)
- Backend store: Azure Database for PostgreSQL (Flexible Server)
- Artifact store: local disk on VM (for simplicity) or Azure Blob Storage (optional)

You’ll run the MLflow server on the VM, then connect from this notebook via its public IP/DNS.


### Connect to the remote MLflow server (from this notebook)
Fill in your VM public IP/DNS below.

**⚠️ Important:** After running the connection cell (Cell 3), if you encounter permission errors when logging artifacts, **restart your Python kernel** and re-run Cell 3. This ensures environment variables are properly set.


In [ ]:
import mlflow

# ⚠️ CRITICAL: Must include http:// scheme for remote server
tracking_uri = "http://52.187.178.215:5000"
mlflow.set_tracking_uri(tracking_uri)
print(f"✓ Tracking URI: {mlflow.get_tracking_uri()}")

In [ ]:
import time
import requests

start = time.time()
response = requests.get("http://52.187.178.215:5000")
latency = time.time() - start
print(f"Server response time: {latency:.2f} seconds")

In [ ]:
# Only get active experiments (faster)
experiments = mlflow.search_experiments(view_type=mlflow.entities.ViewType.ACTIVE_ONLY)

### List existing experiments
Confirm connectivity and see what’s already in the server.


In [ ]:
experiments

### Train a model and log to the remote server
Logs params, metrics, and a model artifact to the Azure-hosted MLflow server.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

### Work with the Model Registry
Create a client pointed at the remote server and register the model.


In [ ]:
from mlflow.tracking import MlflowClient

# Create an MLflow client that connects to your remote tracking server (running on Azure VM)
# This must match the same tracking URI used during logging.
client = MlflowClient(tracking_uri="http://52.187.178.215:5000")

# List all registered models on the server (if any)
client.search_registered_models()

In [ ]:
# Retrieve the latest run from experiment ID "1"
# This assumes your "my-experiment" has ID=1 (you can confirm in the MLflow UI)
run = mlflow.search_runs(experiment_ids=["1"]).iloc[0]

# Extract the unique run ID from that run to reference its logged artifacts
run_id = run.run_id

In [ ]:
# Register the logged model in the MLflow Model Registry
# This creates a new registered model named "azure-iris-classifier" (or updates if it already exists)
# 'model_uri' points to the model artifact inside the specific run's folder
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="azure-iris-classifier"
)

### Notes and troubleshooting
- Ensure Postgres allows connections from the VM and requires SSL.
- If the UI is unreachable, confirm VM NSG/ports and public IP/DNS.
- For artifacts on Azure Blob, ensure the MLflow server has valid credentials and your MLflow version supports the configured scheme.
- Use a managed secret store (e.g., Azure Key Vault) for credentials in production.

**Artifact directory permissions (on VM):**
If you encounter permission errors when the MLflow server tries to write artifacts, ensure the artifact directory is owned by `azureuser`:
```bash
sudo chown -R azureuser:azureuser /home/azureuser/mlruns-artifacts
```
The setup script (`setup-mlflow-server.sh`) handles this automatically, but you may need to run this manually if the directory was created differently.
